In [4]:
import os
import json
from tqdm import tqdm
from glob import glob

dataset_path = "/home/tibia/Documents/LLM/BrainGPT/Dataset/merged"

In [2]:
import os
import shutil
from glob import glob

dataset_path = "/home/tibia/Documents/LLM/BrainGPT/Dataset"
merged_path = os.path.join(dataset_path, "merged")

os.makedirs(merged_path, exist_ok=True)

# Cherche tous les PNG dans tous les sous-dossiers
png_files = glob(os.path.join(dataset_path, "*", "*.png"))

for file_path in png_files:
    # Copie le fichier dans merged en gardant le même nom
    shutil.copy(file_path, merged_path)

print(f"Copied {len(png_files)} files to {merged_path}")

SameFileError: '/home/tibia/Documents/LLM/BrainGPT/Dataset/merged/ID_441cea72_ID_4b13324a92.nii_slice21.png' and '/home/tibia/Documents/LLM/BrainGPT/Dataset/merged/ID_441cea72_ID_4b13324a92.nii_slice21.png' are the same file

In [8]:
def _load_images( image_path: str, num_thread: int) -> dict[str, bytes]:
        """
        Loads the images from the dataset.

        Args:
            image_path (str): The path to the directory containing the dataset images.
            num_threads (int): The number of threads to use for processing the images.

        Returns:
            dict[str, bytes]: A dictionary where the keys are image identifiers and the values are image bytes.
        """
        file_names = glob(os.path.join(image_path, "*"))
        print(f"Found {len(file_names)} files in {image_path}")
        print(f"file_names : {file_names}")  
        
        
        names = set()
        for file_name in file_names:
            # file_name: fullpath
            print(f"file name = :{file_name}")
            image_name = file_name.split("/")[-1].split(".")[0]
            print(f"image_name= :{image_name}")
            id = image_name #image_name: rel_path - .bmp
            names.add(id)
        ids = list(sorted(list(names)))
        print(ids)

        bmps_path = glob(os.path.join(image_path, "*.bmp"))
        jpgs_path = glob(os.path.join(image_path, "*.jpg"))
        jpegs_path = glob(os.path.join(image_path, "*.jpeg"))
        pngs_path = glob(os.path.join(image_path, "*.png"))
        jpgs = set()
        bmps = set()
        pngs = set()
        jpegs = set()
        for path in bmps_path:
            bmps.add(path.split("/")[-1].split(".")[0]) #bmps: rel_path - .bmp
        for path in jpgs_path:
            jpgs.add(path.split("/")[-1].split(".")[0])
        for path in pngs_path:
            pngs.add(path.split("/")[-1].split(".")[0])
        for path in jpegs_path:
            jpegs.add(path.split("/")[-1].split(".")[0])
        
        # print(bmps)

        def get_path(file_name):
            # print(file_name)
            if file_name in jpgs:
                return os.path.join(image_path, file_name + ".jpg")
            elif file_name in pngs:
                # print("file_name", file_name, os.path.join(image_path, file_name + ".png"))
                return os.path.join(image_path, file_name + ".png")
            elif file_name in jpegs:
                return os.path.join(image_path, file_name + ".jpeg")
            elif file_name in bmps:
                return os.path.join(image_path, file_name + ".bmp")

            else:
                # print("===================================", file_name)
                raise Exception("File not found")

        def read_image(file_name) -> bytes:
            with open(file_name, "rb") as f:
                return f.read()

        file_not_found = []

        images = {}
        for id in tqdm(ids, desc="Reading images"):
            i = int(id.split("_")[1])
            if i < 7752:
                try:
                    file_1 = get_path(id)
                    # file_2 = get_path(id + "_2")
                    # print(file_1, file_2)
                    images[id.zfill(5)] = read_image(file_1)
                    # images[id.zfill(5) + "_2"] = read_image(file_2)
                except Exception as e:
                    file_not_found.append(id)
                    # print(f"File not found: {id}")
                    # print(f"Error: {e}")
        return images

images = _load_images(
    image_path=dataset_path,
    num_thread=1
)

Found 72 files in /home/tibia/Documents/LLM/BrainGPT/Dataset/merged
file_names : ['/home/tibia/Documents/LLM/BrainGPT/Dataset/merged/ID_46455_ID_203732_slice18.png', '/home/tibia/Documents/LLM/BrainGPT/Dataset/merged/ID_44172_ID_41332492_slice23.png', '/home/tibia/Documents/LLM/BrainGPT/Dataset/merged/ID_4519652_ID_929185_slice12.png', '/home/tibia/Documents/LLM/BrainGPT/Dataset/merged/ID_46455_ID_203732_slice23.png', '/home/tibia/Documents/LLM/BrainGPT/Dataset/merged/ID_46455_ID_203732_slice22.png', '/home/tibia/Documents/LLM/BrainGPT/Dataset/merged/ID_46455_ID_203732_slice05.png', '/home/tibia/Documents/LLM/BrainGPT/Dataset/merged/ID_46455_ID_203732_slice15.png', '/home/tibia/Documents/LLM/BrainGPT/Dataset/merged/ID_44172_ID_41332492_slice12.png', '/home/tibia/Documents/LLM/BrainGPT/Dataset/merged/ID_46455_ID_203732_slice03.png', '/home/tibia/Documents/LLM/BrainGPT/Dataset/merged/ID_44172_ID_41332492_slice11.png', '/home/tibia/Documents/LLM/BrainGPT/Dataset/merged/ID_4519652_ID_92918

Reading images: 100%|██████████| 72/72 [00:00<00:00, 1911328.41it/s]


In [7]:
import os
from pathlib import Path

dataset_path = Path("/home/tibia/Documents/LLM/BrainGPT/Dataset/merged")

# Liste tous les fichiers png dans le dossier merged
png_files = sorted(dataset_path.glob("*.png"))

for f in png_files:
    old_name = f.name  # ex: ID_441cea72_ID_4b13324a92.nii_slice00.png vient du RSNA

    parts = old_name.split("_")
    if len(parts) < 2:
        continue

    # On garde seulement les chiffres pour la première partie après ID pour que loadimage marche
    first_id = ''.join(filter(str.isdigit, parts[1]))  # '441cea72' -> '44172'

    # On fait meme pour la deuxieme partie
    second_id = ''.join(filter(str.isdigit, parts[3].split(".")[0]))  # '4b13324a92' -> '4132492'

    # Slice et extension
    slice_and_ext = "_".join(parts[4:])  # ex: slice00.png

    # Nouveau nom
    new_name = f"ID_{first_id}_ID_{second_id}_{slice_and_ext}"
    new_path = f.parent / new_name

    print(f"Renaming: {old_name} -> {new_name}")
    f.rename(new_path)

Renaming: ID_441cea72_ID_4b13324a92.nii_slice00.png -> ID_44172_ID_41332492_slice00.png
Renaming: ID_441cea72_ID_4b13324a92.nii_slice01.png -> ID_44172_ID_41332492_slice01.png
Renaming: ID_441cea72_ID_4b13324a92.nii_slice02.png -> ID_44172_ID_41332492_slice02.png
Renaming: ID_441cea72_ID_4b13324a92.nii_slice03.png -> ID_44172_ID_41332492_slice03.png
Renaming: ID_441cea72_ID_4b13324a92.nii_slice04.png -> ID_44172_ID_41332492_slice04.png
Renaming: ID_441cea72_ID_4b13324a92.nii_slice05.png -> ID_44172_ID_41332492_slice05.png
Renaming: ID_441cea72_ID_4b13324a92.nii_slice06.png -> ID_44172_ID_41332492_slice06.png
Renaming: ID_441cea72_ID_4b13324a92.nii_slice07.png -> ID_44172_ID_41332492_slice07.png
Renaming: ID_441cea72_ID_4b13324a92.nii_slice08.png -> ID_44172_ID_41332492_slice08.png
Renaming: ID_441cea72_ID_4b13324a92.nii_slice09.png -> ID_44172_ID_41332492_slice09.png
Renaming: ID_441cea72_ID_4b13324a92.nii_slice10.png -> ID_44172_ID_41332492_slice10.png
Renaming: ID_441cea72_ID_4b13324